In [1]:
# boas práticas: criar uma célula com todas as bibliotecas necessárias
# importando as bibliotecas necessárias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Turmas 2017.2
# http://dados.ufrn.br/dataset/turmas

# turmas
turmas_url = 'http://bit.do/turmas_ufrn_2017_2'

# matrículas em componentes em 2017.2
notas_url = 'http://bit.do/notas_ufrn_2017_2'

# componentes curriculares presenciais
disciplinas_url = 'http://bit.do/componentes_curriculares_ufrn'

# docentes da ufrn atualizado em 2018
docentes_url = 'http://bit.do/docentes_ufrn_2018'

# turmas 2017.2
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes em 2017.2
notas = pd.read_csv(notas_url, sep=';')

# componentes curriculares presenciais
# por alguma razão ou outra, esse dataset tinha algum lixo em linhas que ocasionava 
# erros de leitura, dessa forma foi realizado um filtro apenas por colunas mais importantes
disciplinas = pd.read_csv(disciplinas_url, 
                          sep=';', 
                          usecols=["id_componente","codigo","nivel","nome","unidade_responsavel"])

docentes = pd.read_csv(docentes_url, sep=';')

In [4]:
# Adaptando o dataframe turma e disciplinas para o merge
# Esse procedimento é necessário pois o dataframe turma está codificado com ids

# A coluna id_componente precisa ser a mesma em ambos os dataframes
turmas.rename(columns={'id_componente_curricular': 'id_componente'}, inplace=True)

# id_componente em ambos os dataframes precisam ser do mesmo tipo
turmas.id_componente = turmas.id_componente.astype(str)

# Limpando o dataframe disciplinas com linhas desnecessárias
disciplinas.dropna(inplace=True)
disciplinas = disciplinas[~disciplinas.id_componente.duplicated()]

In [5]:
# Fazendo um merge entre o dataframe turmas e disciplinas utilizando como base a coluna id_componente
# https://pandas.pydata.org/pandas-docs/stable/merging.html
colunas = ["id_turma","id_docente_interno","codigo","nivel_ensino","nome","unidade_responsavel",
          "situacao_turma","capacidade_aluno","descricao_horario"]

# merge os dois dataframes utilizando como base a coluna id_componenteb
merge_turmas_disciplinas = pd.merge(turmas, disciplinas, on='id_componente')[colunas]

merge_turmas_disciplinas.dropna(inplace=True)

In [6]:
# filtrando as turmas com código IMD
turmas_imd = merge_turmas_disciplinas[merge_turmas_disciplinas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# Adaptar o nome da coluna id_docente_interno para id_servidor
turmas_imd.rename(columns={'id_docente_interno': 'id_servidor'}, inplace=True)

turmas_imd.capacidade_aluno = turmas_imd.capacidade_aluno.astype(np.int64)

# Adaptar os tipos de dados em turmas_imd para o merge com o dataframe docentes
turmas_imd.id_servidor = turmas_imd.id_servidor.astype(np.int64)

In [7]:
# merge os dataframes  turmas_imd e docentes utilizando como base a coluna id_servidor

colunas = ["id_turma", "codigo", "nivel_ensino", "nome_y", "codigo", "nivel_ensino",
          "nome_x","capacidade_aluno", "descricao_horario"]
turmas_imd_nomesprof = pd.merge(turmas_imd, docentes, on='id_servidor')[colunas]
turmas_imd_nomesprof

,id_turma,codigo,nivel_ensino,nome_y,codigo,nivel_ensino,nome_x,capacidade_aluno,descricao_horario
0,57608376,IMD0017,GRADUAÇÃO,LUCELIO DANTAS DE AQUINO,IMD0017,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS I,60,5N34 (24/07/2017 - 16/12/2017)
1,57608398,IMD0027,GRADUAÇÃO,LUCELIO DANTAS DE AQUINO,IMD0027,GRADUAÇÃO,PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II,60,3N34 (24/07/2017 - 16/12/2017)
2,57608691,IMD0335,GRADUAÇÃO,LUCELIO DANTAS DE AQUINO,IMD0335,GRADUAÇÃO,TÓPICOS ESPECIAIS I,40,35T56 (24/07/2017 - 16/12/2017)
3,57608377,IMD0019,GRADUAÇÃO,LOURENA KARIN DE MEDEIROS ROCHA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)
4,57608377,IMD0019,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,65,246M3456 (24/07/2017 - 16/12/2017)
5,57608435,IMD0522,GRADUAÇÃO,CHARLES ANDRYE GALVAO MADEIRA,IMD0522,GRADUAÇÃO,JOGOS DIGITAIS COMO FERRAMENTA PEDAGÓGICA,44,24T56 (24/07/2017 - 16/12/2017)
6,57608378,IMD0019,GRADUAÇÃO,ANDRE LUIZ DA SILVA SOLINO,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,67,246N1234 (24/07/2017 - 16/12/2017)
7,57608385,IMD0012.0,GRADUAÇÃO,ANDRE LUIZ DA SILVA SOLINO,IMD0012.0,GRADUAÇÃO,INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO,44,24N12 (24/07/2017 - 16/12/2017)
8,57608391,IMD0012.1,GRADUAÇÃO,ANDRE LUIZ DA SILVA SOLINO,IMD0012.1,GRADUAÇÃO,PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO,41,6N12 (24/07/2017 - 16/12/2017)
9,57608378,IMD0019,GRADUAÇÃO,AUGUSTO JORGE DE MACEDO,IMD0019,GRADUAÇÃO,RESOLUÇÃO DE PROBLEMAS MATEMÁTICOS PARA TI,67,246N1234 (24/07/2017 - 16/12/2017)


In [8]:
# filtrar apenas as turmas do IMD
notas_imd = notas[notas.id_turma.isin(turmas_imd_nomesprof.id_turma)]

# merge entre os dataframes notas_imd e turmas_imd_nomesprof utilizando como referencia id_turma
notas_imd_merge = pd.merge(notas_imd, turmas_imd_nomesprof, on='id_turma')
notas_imd

,id_turma,discente,id_curso,unidade,nota,reposicao,faltas_unidade,media_final,numero_total_faltas,descricao
85479,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,3.0,0.0,f,0.0,1.3,0.0,CANCELADO
85480,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,2.0,0.0,f,0.0,1.3,0.0,CANCELADO
85481,57608383,8488e52d4ca947f5504905cb261636ca,92127264.0,1.0,3.9,f,0.0,1.3,0.0,CANCELADO
85482,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,2.0,0.0,f,0.0,1.8,0.0,CANCELADO
85483,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,1.0,5.3,f,0.0,1.8,0.0,CANCELADO
85484,57608383,6979f555ed7b878d55f9cd3fa4e5c032,92127264.0,3.0,0.0,f,0.0,1.8,0.0,CANCELADO
85485,57608383,e68aa00afbec6ea1d2511386edeaf5d0,92127264.0,3.0,0.0,f,0.0,1.7,0.0,CANCELADO
85486,57608383,e68aa00afbec6ea1d2511386edeaf5d0,92127264.0,2.0,0.0,f,0.0,1.7,0.0,CANCELADO
85487,57608383,e68aa00afbec6ea1d2511386edeaf5d0,92127264.0,1.0,5.0,f,0.0,1.7,0.0,CANCELADO
85488,57608383,b995ab09e808edef4e250508e2514edf,92127264.0,3.0,5.0,f,0.0,4.5,0.0,CANCELADO


In [9]:
# todas as disciplinas em 2017.2
notas_imd_merge.nome_x.unique()

array(['INTRODUÇÃO ÀS TÉCNICAS DE PROGRAMAÇÃO',
       'PRÁTICAS DE TÉCNICAS DE PROGRAMAÇÃO', 'DESIGN DE SOFTWARE II',
       'INTRODUÇÃO À GOVERNANÇA E GESTÃO DE TI', 'ACESSIBILIDADE DIGITAL',
       'EMPREENDEDORISMO',
       'PROCESSO DE DESENVOLVIMENTO DE SOFTWARE EDUCACIONAL',
       'TÓPICOS ESPECIAIS I', 'INTRODUÇÃO AO TESTE DE SOFTWARE',
       'PROGRAMAÇÃO CONCORRENTE E DISTRIBUÍDA',
       'RELAÇÕES ENTRE COMUNICAÇÃO, MÍDIA E EDUCAÇÃO',
       'CÁLCULO DIFERENCIAL E INTEGRAL I', 'SISTEMAS OPERACIONAIS',
       'REDES DE COMPUTADORES', 'VETORES E GEOMETRIA ANALÍTICA',
       'JOGOS DIGITAIS COMO FERRAMENTA PEDAGÓGICA',
       'DESENVOLVIMENTO DE SISTEMAS WEB I',
       'PROTOCOLO E SERVIÇOS DE INTERNET',
       'PROJETOS BASEADOS EM MICROCONTROLADORES',
       'PRÁTICAS DE LEITURA E ESCRITA EM PORTUGUÊS II',
       'TÓPICOS ESPECIAIS EM INFORMÁTICA EDUCACIONAL B"',
       'TÓPICOS ESPECIAIS EM INFORMÁTICA EDUCACIONAL C"',
       'TÓPICOS ESPECIAIS EM INFORMÁTICA EDUCACIONAL D"

In [76]:
dados = notas_imd_merge[notas_imd_merge.nome_x == 'FUNDAMENTOS MATEMÁTICOS DA COMPUTAÇÃO II']
dados = dados[~dados.discente.duplicated()]
mypivot = dados.pivot_table(index=["descricao", "numero_total_faltas"], values="nome_x", aggfunc="count")
mypivot

nome_x
descricao                        numero_total_faltas        
APROVADO                         14.0                      1
                                 30.0                      1
APROVADO POR NOTA                0.0                       1
                                 2.0                       2
                                 4.0                       2
                                 6.0                       2
                                 8.0                       1
                                 10.0                      1
                                 12.0                      1
                                 14.0                      2
                                 20.0                      1
CANCELADO                        0.0                       2
                                 4.0                       1
                                 22.0                      1
                                 30.0                      1
                                 36.0                      1
                                 54.0                      1
                                 56.0                      1
                                 62.0                      1
                                 64.0                      1
REPROVADO                        6.0                       1
                                 10.0                      1
                                 12.0                      1
                                 16.0                      1
                                 20.0                      2
                                 22.0                      3
                                 24.0                      1
                                 26.0                      1
                                 30.0                      1
                                 31.0                      2
REPROVADO POR MÉDIA E POR FALTAS 28.0                      1
                                 30.0                      1
                                 32.0                      1
                                 34.0                      1
                                 38.0                      1
                                 40.0                      2
                                 42.0                      2
                                 44.0                      1
                                 46.0                      1
                                 48.0                      2
                                 50.0                      3
                                 52.0                      1
                                 54.0                      2
                                 58.0                      2
                                 64.0                      2
                                 66.0                      4